## CSE 258 Web Mining and Recommender Systems
### Assignment 1 
**Shivani Bhakta** 

In [1]:
import gzip
from collections import defaultdict
import math
import scipy
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
def assertFloat(x):
    assert type(float(x)) == float

def assertFloatList(items, N):
    assert len(items) == N
    assert [type(float(x)) for x in items] == [float]*N

In [4]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [5]:
def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        u,b,r = l.strip().split(',')
        r = int(r)
        yield u,b,r

In [6]:
allRatings = []
for l in readCSV("assignment1/train_Interactions.csv.gz"):
    allRatings.append(l)
# len(allRatings) # 200000

In [7]:
#Note: ratingsTrain = list of (user id, book id,rating)
ratingsTrain = allRatings[:190000]
ratingsValid = allRatings[190000:]
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
ratingDict = defaultdict(float)
for u,b,r in ratingsTrain:
    ratingsPerUser[u].append((b,r))
    ratingsPerItem[b].append((u,r))

# this needs to be done for entire dataset 
for u,b,r in allRatings:
    ratingDict[(u,b)] = float(r)
    

len(ratingsTrain), len(ratingsValid)

(190000, 10000)

## Rating Prediction

In [8]:
labels = [d[-1] for d in ratingsTrain]
ratingMean = sum([d[-1] for d in ratingsTrain]) / len(ratingsTrain)
alwaysPredictMean = [ratingMean for d in ratingsTrain]

In [9]:
N = len(ratingsTrain)
nUsers = len(ratingsPerUser)
nItems = len(ratingsPerItem)
users = list(ratingsPerUser.keys())
items = list(ratingsPerItem.keys())

In [10]:
alpha = ratingMean
userBiases = defaultdict(float)
itemBiases = defaultdict(float)
userGamma = {}
itemGamma = {}

In [11]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [12]:
def inner(x, y):
    return sum([a*b for a,b in zip(x,y)])

In [13]:
def prediction(user, item):
    return alpha + userBiases[user] + itemBiases[item] + inner(userGamma[user], itemGamma[item])

In [14]:
def unpack(theta):
    global alpha
    global userBiases
    global itemBiases
    global userGamma
    global itemGamma
    index = 0
    alpha = theta[index]
    index += 1
    userBiases = dict(zip(users, theta[index:index+nUsers]))
    index += nUsers
    itemBiases = dict(zip(items, theta[index:index+nItems]))
    index += nItems
    for u in users:
        userGamma[u] = theta[index:index+K]
        index += K
    for i in items:
        itemGamma[i] = theta[index:index+K]
        index += K

In [15]:
def cost(theta, labels, lamb):
    unpack(theta)
    predictions = [prediction(d[0], d[1]) for d in ratingsTrain]
    cost = MSE(predictions, labels)
    print("MSE = " + str(cost))
    for u in users:
        cost += lamb*userBiases[u]**2
        for k in range(K):
            cost += lamb*userGamma[u][k]**2
    for i in items:
        cost += lamb*itemBiases[i]**2
        for k in range(K):
            cost += lamb*itemGamma[i][k]**2
    return cost

In [16]:
def derivative(theta, labels, lamb):
    unpack(theta)
    N = len(ratingsTrain)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dItemBiases = defaultdict(float)
    dUserGamma = {}
    dItemGamma = {}
    for u in ratingsPerUser:
        dUserGamma[u] = [0.0 for k in range(K)]
    for i in ratingsPerItem:
        dItemGamma[i] = [0.0 for k in range(K)]
    for d in ratingsTrain:
        u,i = d[0], d[1]
        pred = prediction(u, i)
        diff = pred - d[-1]
        dalpha += 2/N*diff
        dUserBiases[u] += 2/N*diff
        dItemBiases[i] += 2/N*diff
        for k in range(K):
            dUserGamma[u][k] += 2/N*itemGamma[i][k]*diff
            dItemGamma[i][k] += 2/N*userGamma[u][k]*diff
    for u in userBiases:
        dUserBiases[u] += 2*lamb*userBiases[u]
        for k in range(K):
            dUserGamma[u][k] += 2*lamb*userGamma[u][k]
    for i in itemBiases:
        dItemBiases[i] += 2*lamb*itemBiases[i]
        for k in range(K):
            dItemGamma[i][k] += 2*lamb*itemGamma[i][k]
    dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dItemBiases[i] for i in items]
    for u in users:
        dtheta += dUserGamma[u]
    for i in items:
        dtheta += dItemGamma[i]
    return numpy.array(dtheta)

In [17]:
MSE(alwaysPredictMean, labels)

1.741577477865528

In [59]:
K = 2
scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + # Initialize alpha
                            [0.0]*(nUsers+nItems) + # Initialize beta
                            [random.random() * 0.1 - 0.05 for k in range(K*(nUsers+nItems))], # Gamma
                            derivative, args = (labels, 0.001), maxiter=150)

MSE = 1.7446630867789088
MSE = 2.5694495595399878
MSE = 1.7412624259878473
MSE = 1.7409419845722747
MSE = 1.7396672007870977
MSE = 1.7346797755522285
MSE = 1.7165174250357438
MSE = 1.6392822008954777
MSE = 1.6257389922202112
MSE = 1.6230743132681769
MSE = 1.6248991988550516
MSE = 1.625427748270238
MSE = 1.626003397460628
MSE = 1.6261376116402422
MSE = 1.626125291412492


(array([ 3.63133758e+00, -2.28919326e-01,  1.16211020e-02, ...,
        -2.13735385e-07,  1.72138012e-06, -3.72062116e-07]),
 1.6710666994948196,
 {'grad': array([ 7.80222495e-08, -3.96446999e-07,  1.38334241e-08, ...,
         -4.15399010e-10,  3.45719799e-09, -7.42832694e-10]),
  'task': 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL',
  'funcalls': 15,
  'nit': 10,
  'warnflag': 0})

In [60]:
ypred = []
valid_labels = []
for user, item, r in ratingsValid:
    valid_labels.append(r)
    
    if (user in userBiases) and (item in itemBiases) and (user in userGamma) and (item in itemGamma):
        ypred.append(prediction(user, item))
    else: ypred.append(0)

print("MSE = %f" % MSE(ypred, valid_labels))
validMSE = MSE(ypred, valid_labels)

MSE = 1.632502


In [61]:
predictions = open("predictions_Rating.csv", 'w')
for l in open("assignment1/pairs_Rating.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,b = l.strip().split(',')
    bu = 0
    bi = 0
    gu = []
    gi = []
    if u in userBiases:
        bu = userBiases[u]
    if b in itemBiases:
        bi = itemBiases[b]
    if u in userGamma: 
        gu = userGamma[u]
    if b in itemGamma: 
        gi = itemGamma[b]
#     print(bu,bi,gu,gi,inner(gu,gi))
    _ = predictions.write(u + ',' + b + ',' + str(alpha + bu + bi + inner(gu,gi)) + '\n')
    ## here I recommend printing alpha bu bi and inner(gu,gi)
predictions.close()


## Read prediction Task     
Predict given a (user,book) pair from ‘pairs Read.csv’ whether the user would read the book (0 or 1). Accuracy will be measured in terms of the categorization accuracy (fraction of correct predictions). The test set has been constructed such that exactly 50% of the pairs correspond to read books and the other 50% do not.


** Plan **
- Make a list of users and the books they have read  # done 
- Find other users who have read similar/same book as this user, and see what other books they have read
-  

In [ ]:
# Generate a negative set

userSet = set()
bookSet = set()
readSet = set()

for u,b,r in allRatings:
    userSet.add(u) # unique users 
    bookSet.add(b) # unique books 
    readSet.add((u,b)) #unique pairs 

lUserSet = list(userSet) # all users in entire dataset 
lBookSet = list(bookSet) # all books in entire dataset 

notRead = set() # this is a set of negative pairs. 
for u,b,r in ratingsValid:
    #u = random.choice(lUserSet)
    b = random.choice(lBookSet)
    while ((u,b) in readSet or (u,b) in notRead):
        b = random.choice(lBookSet)
    notRead.add((u,b))

# readValid = set()
# for u,b,r in ratingsValid:
#     readValid.add((u,b))
ratingsValid = [[d[0],d[1],d[2],1] for d in ratingsValid]
for u,b in notRead: 
    ratingsValid.append([u,b,0,0])
print(len(ratingsValid))
print(len(userSet))
print(len(bookSet))

In [ ]:
def getMostPopular():
    bookCount = defaultdict(int)
    totalRead = 0

    for user,book,_ in readCSV("assignment1/train_Interactions.csv.gz"):
        bookCount[book] += 1
        totalRead += 1
        
        
    mostPopular = [(bookCount[x]/len(userSet), x) for x in bookCount]
    mostPopular.sort()
    mostPopular.reverse()
    return mostPopular


In [ ]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom > 0:
        return numer/denom
    return 0

# taken from the workbook of chapter 4
def CosineSet(s1, s2):
    # Not a proper implementation, operates on sets so correct for interactions only
    numer = len(s1.intersection(s2))
    denom = math.sqrt(len(s1)) * math.sqrt(len(s2))
    if denom == 0:
        return 0
    return numer / denom

In [ ]:
def Cosine_btwn_users(u1, u2, booksPerUser, ratingDict):
    '''
     Compute similarity Between two users based on the ratings 
     they gave to the books they both have read
    '''
    # inter contains the books both users have read 
    inter = booksPerUser[u1].intersection(booksPerUser[u2])
    numer = 0
    denom1 = 0
    denom2 = 0
    
    for book in inter:
        numer += ratingDict[(u1,b)]*ratingDict[(u2,b)]
        
    for book in booksPeruser[u1]:
        denom1 += ratingDict[(u1,book)]**2
        
    for book in booksPeruser[u2]:
        denom2 += ratingDict[(u2,book)]**2
        
    denom = math.sqrt(denom1) * math.sqrt(denom2)
    
    if denom == 0: return 0
    
    return numer / denom

def get_mostSimilar_user(user, book, booksPerUser, usersPerBook, ratingDict ): 
    '''
    for given user,book pair
    return the most similar user v for the pair
    '''
    similarities = []
    users = set(usersPerBook[book])  # all users for given book
    
    for u2 in users:
        if u2 == user: continue
        sim = Cosine_btwn_users(user, u2, booksPerUser, ratingDict)
        similarities.append(sim)
    similarities.sort(reverse=True)
    return similarities[0]   

In [ ]:
def weighted_cosine_between_users(u1, u2):
    # Between two items
    booksPerUser = user_book
    inter = set(booksPerUser[u1]).intersection(set(booksPerUser[u2]))
    numer = 0
    denom1 = 0
    denom2 = 0
    for b in inter:
        numer += ratingDict[(u1,b)]*ratingDict[(u2,b)]
    for b in booksPerUser[u1]:
        denom1 += ratingDict[(u1,b)]**2
    for b in booksPerUser[u2]:
        denom2 += ratingDict[(u2,b)]**2
    denom = math.sqrt(denom1) * math.sqrt(denom2)
    if denom == 0: return 0
    return numer / denom

def weightedUsersCosineSim(user,book):
    sims = [0]
    users = set(book_user[book]) # all users for book
    for user_x in users:
        if(user_x == user):
            continue
        sim = weighted_cosine_between_users(user_x,user)
        sims.append(sim)
    return max(sims)


def weightedBooksCosineSim(user,book):
    sims = [0]
    books = set(user_book[user]) # all books for user
    for book2 in books:
        if(book2 == book):
            continue
        sim = weighted_cosine_between_books(book2,book)
        sims.append(sim)
    return max(sims)

In [ ]:
def generateFeatures(user, book, booksPerUser,usersPerBook): 
    
    features = [1]
    mostPopular = getMostPopular()
    features.append(mostPopular[0])
    
    
    f3 = Cosine(u1, u2, booksPerUser, ratingDict)
    
    
    return features

In [ ]:
def generate_features(user,book):
    features = [1,bookCount[book]/len(userSet)]
    features.append(maxUserSim(user,book,Jaccard))
    features.append(maxBookSim(user,book,Jaccard))
    
#     features.append(maxUserSim(user,book,CosineSet))
#     features.append(maxBookSim(user,book,CosineSet))
    
#     features.append(weightedUsersCosineSim(user,book))
#     features.append(weightedBooksCosineSim(user,book))


    return features

In [ ]:
def generateData(data):
    X,y = [],[]
    for user,book,rating,isRead in tqdm(data):
        features = generate_features(user,book)
        X.append(features)
        y.append(isRead)
    return X,y

In [ ]:
# get all the books user has read
# get all the users who read the book 
booksPerUser = defaultdict(list)
usersPerBook = defaultdict(list)
for u,b,r in allRatings:
    booksPerUser[u].append(b)
    usersPerBook[b].append(u)

# 27945
# 6688
print(len(usersPerBook)) 
print(len(booksPerUser))

In [ ]:
mostPopular = getMostPopular()
mostPopular[0]

In [ ]:
# Run on test set

In [ ]:
data_test = []

for d in readGz("assignment1/test_Category.json.gz"):
    data_test.append(d)

In [ ]:
Xtest = [feature(d) for d in data_test]
pred_test = mod.predict(Xtest)

In [ ]:
predictions = open("predictions_Category.csv", 'w')
pos = 0

for l in open("assignment1/pairs_Category.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,b = l.strip().split(',')
    _ = predictions.write(u + ',' + b + ',' + str(pred_test[pos]) + '\n')
    pos += 1

predictions.close()

In [ ]:
f = open("answers_hw3.txt", 'w')
f.write(str(answers) + '\n')
f.close()